# Computing the Euclidean Distance Matrix with NumPy

In this notebook we implement two functions to compute the Euclidean distance matrix. We use a simple algebra trick that makes possible to write the function in a completely vectorized way in terms of optimized NumPy functions.

In [ ]:
import numpy as np

Now we know bradcasting. Let's use it to implement a function that calculates the Euclidean distance matrix of an array of vectors.

In [ ]:
def euclidean_broadcast(x, y):
    """Euclidean square distance matrix.
    
    Inputs:
    x: (N, m) numpy array
    y: (N, m) numpy array
    
    Ouput:
    (N, N) Euclidean square distance matrix:
    r_ij = (x_ij - y_ij)^2
    """
    diff = x[:, np.newaxis, :] - y[np.newaxis, :, :]

    return (diff * diff).sum(axis=2)

<mark>**Question**</mark>: At this point you are starting to get acquainted with the `numpy.ndarray`s and its memory managment. Could you analyse the advantages and possible drawbacks of the `euclidean_broadcast` function? Write a positive and a negative point about it.

***

Let's consider a more sophisticated implementation:

In [ ]:
def euclidean_trick(x, y):
    """Euclidean square distance matrix.
    
    Inputs:
    x: (N, m) numpy array
    y: (N, m) numpy array
    
    Ouput:
    (N, N) Euclidean square distance matrix:
    r_ij = (x_ij - y_ij)^2
    """
    x2 = np.einsum('ij,ij->i', x, x)[:, np.newaxis]
    y2 = np.einsum('ij,ij->i', y, y)[np.newaxis, :]

    xy = x @ y.T

    return np.abs(x2 + y2 - 2. * xy)

## The `euclidean_trick` function

Each element of the Euclidean distance matrix is the scalar product of the difference between two rows of the array. `euclidean_trick` takes advantage of this by doing the following
$$
\sum_k {(x_{ik}-y_{ik})^2} = (\vec{x}_i - \vec{y}_j)\cdot(\vec{x}_i - \vec{y}_j) = \vec{x}_i\cdot\vec{x}_i + \vec{y}_j\cdot\vec{y}_j - 2\vec{x}_i\cdot\vec{y}_j
$$

Fortunately, there are NumPy functions to compute each of these terms:

$\vec{x}_i\cdot\vec{y}_j$ $\rightarrow$ `x @ y` : Matrix product of $\{\vec{x}\}$ and $\{\vec{y}\}$

$\vec{x}_i\cdot\vec{x}_i$ $\rightarrow$ `np.einsum('ij,ij->i', x, x)[:, np.newaxis]` : A $(n,1)$ vector of elements $\sum_j x_{ij}x_{ij}$

$\vec{y}_j\cdot\vec{y}_j$ $\rightarrow$ `np.einsum('ij,ij->i', y, y)[np.newaxis, :]` : A $(1,n)$ vector of elements $\sum_j y_{ij}y_{ij}$

To have all the combinations $ij$ of the sum $\vec{x}_i\cdot\vec{x}_i + \vec{y}_j\cdot\vec{y}_j$, we add a new axis to each of the arrays, transpose one them and add them.

Let's see now how the `np.einsum` function works. `einsum` stands for Einstein summation, which is used in tensor algebra to write compact expressions without the sum symbol ($\sum$). Within the Einstein summation notation, whenever there are repeated indexes, there is a sum over them. For instance, the expression
$$x_{ik}y_{kj}$$
is equivalent to
$$\sum_k x_{ik}y_{kj}$$

`np.einsum` uses a generalized form of the Einstein summation by adding the symbol `->` to prevent summing over certain indexes. The specific operation we use here, `np.einsum('ij,ij->i', x, x)`, gives the vector
$$
\begin{bmatrix}
\sum_k x_{1k}x_{1k} \\
\sum_k x_{2k}x_{2k} \\
 ...                \\
\sum_k x_{nk}x_{nk} \\
\end{bmatrix}
$$
Note that the resulting vector is represented here as a column vector just for visualization purposes. It's is an `(n,)` NumPy array.

Let's check now step-by-step what the `euclidean_trick` function does:

In [ ]:
# Lets generate some random data
nsamples = 10
nfeat = 3

rng = np.random.default_rng()
x = rng.random((nsamples, nfeat))
x

In [ ]:
x2 = np.einsum('ij,ij->i', x, x)
x2.shape

In [ ]:
x2 = np.einsum('ij,ij->i', x, x)[:, np.newaxis]
x2.shape

In [ ]:
(x2 + x2.T).shape

We now use `x @ x.T` to perform the matrix multiplication of the full dataset by itself. We didn't use it before as alternative to `np.einsum` because it doesn't perform row by row scalar products. Instead `x @ x.T`, which calls `np.matmul`, expects two arrays with matching shapes $(m,n)$ and $(n,m)$ to perform a matrix multiplication.

We could have used `np.einsum('ik,jk', x, x)` to perform the matrix multiplication, but we chose `x @ x.T` instead. This is because `x @ x.T` is a very sophisticated too, plus it uses OpenMP threads. This results in a very fast execution.

You are welcome to time them and look at the `top` command to see how `x @ x.T` uses multiple OpenMP threads.

In [ ]:
xy = x @ x.T
xy.shape

Now, considering that the reason we are using `np.einsum` is to get rid of the loops, why didn't we use something like `(x*x).sum(axis=1)`? Let's run the next cell comparing them:

In [ ]:
# let's use a larger array for timing the function calls
nsamples = 1000
nfeat = 300

x = 10. * rng.random((nsamples, nfeat))

# it gives the same result
np.abs(np.einsum('ij,ij->i', x, x) - (x*x).sum(axis=1)).max()

# but it's not as fast as `np.einsum`
%timeit np.einsum('ij,ij->i', x, x)
%timeit (x * x).sum(axis=1)

Doing a reduction with the ufunc `np.add` is also slower than `np.einsum`

In [ ]:
%timeit np.add.reduce(x * x, axis=1)

# As homework, check what's the meaning of the previous line!

Finally, let's time both implementations and check that they give the same result!

In [ ]:
nsamples = 2000
nfeat = 50

x = 10. * rng.random((nsamples, nfeat))

%timeit euclidean_broadcast(x, x)
%timeit euclidean_trick(x, x)

Let's check that both implementations give the same results:

In [ ]:
np.abs(euclidean_broadcast(x, x) - euclidean_trick(x, x)).max()

Another way to check:

In [ ]:
False in np.isclose(euclidean_broadcast(x, x), euclidean_trick(x, x))

<mark>**Homework**</mark> Change the implementation of `euclidean_broadcast` function to make faster using `einsum` to do the final sum. How much is the speed-up? Compare it with both the original `euclidean_broadcast` and `euclidean_trick`. Check that the result is the same!

In [ ]:
%load solutions/edm-broadcast-einsum.py

# Conclusions

The main points to take from this notebook are:
  * NumPy is all about vectorization. Loops in python must be avoided.
  * Always consider different vectorized implementations and compare them.
  * Even within NumPy, some functions might bring a more significant speedup than others.